# Capstone Project 

Twitter sentiment anaylsis to predict twitter sentiment analysis

# Obtain 

In [1]:
import numpy as np
import pandas as pd
import re 
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
import matplotlib.pyplot as mlpt
import tweepy
import sys
sys.path.append("twint/")
!pip install --user -r requirements.txt
!pip install --upgrade --user -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("twint/")
import twint 
import nest_asyncio
import warnings
warnings.filterwarnings('ignore')
nest_asyncio.apply()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/afeinberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
You should consider upgrading via the '/Users/afeinberg/anaconda3/envs/learn-env/bin/python -m pip install --upgrade pip' command.
Obtaining twint from git+https://github.com/twintproject/twint.git@origin/master#egg=twint
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/afeinberg/anaconda3/envs/learn-env/lib/python3.6/site-packages/pip/_internal/vcs/git.py", line 293, in get_remote_url
    found_remote = remotes[0]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/afeinberg/anaconda3/envs/learn-env/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 188, in _main
    status = self.run(options, args)
  File "/Users/afeinberg/anaconda3/envs/learn-env/lib/python3.6/site-packages/pip/_internal/cli/req_command.py", line 185, in wrapper
    return func(self

### Using Twitter API

I've recieved access to the twitter API, however I soon realized that the API only offers you 7 days of historical tweets. This is not enough data to predict stock price. Leaving the process and functions I used to get to this source of data, with the purpose of identifying it's limitations. 

Removed credentials for security purposes

In [2]:
consumer_key = ###########
consumer_secret = ########
access_token = ##########
access_token_secret = #########

In [3]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 
 

In [4]:
def get_result(company_name,company_ticker,no_of_tweets):

    tweets = tweepy.Cursor(api.search,q=(company_name,company_ticker),lang="en").items(no_of_tweets) #Getting no_of_tweets with the car_name
    
    all_tweets = [] 
    all_date = []
    all_favourite_count = []
    all_retweet_count = []
    all_image_count = []
    
    
    #Iterating through all the tweets
    for tweet in tweets:
        image_counter=0

        all_tweets.append(tweet.text) #Getting the text of the tweet
        all_date.append(tweet.created_at) #Getting the date of the tweet
        all_favourite_count.append(tweet.favorite_count) #Getting the number of likes of the tweet
        all_retweet_count.append(tweet.retweet_count) #Getting the number of retweets of the tweet


        if 'media' in tweet.entities:  #Getting the number of images attached with the tweet
            for image in  tweet.entities['media']:
                image_counter = image_counter+1
            all_image_count.append(image_counter) 
        else :
            all_image_count.append(str(None))

        

    tweets = pd.DataFrame(list(zip(all_tweets, all_date,all_favourite_count,all_retweet_count,all_image_count)), columns =['Tweet', 'Date','No. of Likes','No. of Retweets','No. of Images'])
    return tweets #Returns the output in a tabular format

In [5]:
tweets = get_result('Apple','aapl',100)
tweets.tail()
#Function to get the Final Results. 

,Tweet,Date,No. of Likes,No. of Retweets,No. of Images
95,The fabled tech antitrust hearing occurred ear...,2020-07-31 20:26:54,0,0,None
96,RT @appleinsider: #Apple has overtaken oil gia...,2020-07-31 20:26:40,0,27,None
97,RT @11Graphs: 52w return (%) among largest #st...,2020-07-31 20:26:38,0,2,None
98,RT @CNBCnow: BREAKING: Apple announces 4-for-1...,2020-07-31 20:26:28,0,718,1
99,RT @CNBC: Stocks rise after Big Tech crushes e...,2020-07-31 20:26:23,0,8,None


### Using Twint for Twitter

Twint utilizes Twitter’s search operators to let you scrape Tweets from specific users, and scrape Tweets relating to certain topics. It is the route we need to take as it allows you to pull a large volume of tweets and can access more outdated tweets. This will be helpful in building a predictive time series consisting of 30 days of tweets. 

In [6]:
c = twint.Config()
c.Search = 'Apple', 'aapl'
c.Since = '2019-01-01'
c.Until = '2019-02-28'
c.Limit = 1000
c.Store_csv = True
c.Pandas = True
c.Replies = True
c.Lang = 'en'
c.Count = True
c.Format = "Tweet id: {id} | Date: {date} | Time: {time} | Tweet: {tweet} | Replies: {replies}"
c.Custom['tweet'] = ['id', 'date', 'time', 'tweet','replies']
twint.run.Search(c) 

Tweet id: 1100848143451807745 | Date: 2019-02-27 | Time: 15:00:24 | Tweet: Official iOS 12 adoption stats as published by Apple through February 24, 2019. $AAPL #Peerless pic.twitter.com/uF9f4SeS8y | Replies: 1
Tweet id: 1100756422541766657 | Date: 2019-02-27 | Time: 08:55:56 | Tweet: Haha your not very informed it’s RIP $AAPL not $QCOM based on that trend you presented. #Qualcomm revenue will rise off this bottom with their chips, and modems fueling the Chinese OEMs recent growth into global markets. #APPLE isn’t a customer anymore using inferior $intc. | Replies: 1
Tweet id: 1100745238316965888 | Date: 2019-02-27 | Time: 08:11:30 | Tweet: @FTC with #Huawei & #Apple as a star witness wants Judge Koh to free 6 billion Non-Americans from paying royalties for American 5G Innovation. Suits Huawei just fine don’t have China government stabbing them in the back! #qualcomm $qcom @POTUS @DOJPH #ftcqcom $mu $ge $aapl $intc https://twitter.com/newtgingrich/status/1100733935162716160 … | Replies

Tweet id: 1099018607684710402 | Date: 2019-02-22 | Time: 13:50:29 | Tweet: content that users create on Apple's Mac's, iPads, iPhones, etc.  Plus it gives Twitter the resources they need to continue to give everyone a voice but also to innovate faster, rely less on ads, improve the conversational health and make the platform more useful. $TWTR $AAPL  pic.twitter.com/bs9resNRyr | Replies: 1
Tweet id: 1098921085582614528 | Date: 2019-02-22 | Time: 07:22:58 | Tweet: What we witnessed with Apple vs General Electric over the last decade was "Creative Destruction" at it's finest. $GE could have bought $AAPL for $2 billion USD in '96...  pic.twitter.com/k4uc8mtH84 | Replies: 0
Tweet id: 1098834938713894912 | Date: 2019-02-22 | Time: 01:40:39 | Tweet: Growing global wealth/income disparity crushing demand for G&S ergo CapX. Wealth/income moves to lower propensity to consume classes = more savings for spec.  Instead of buying apples, they buy AAPL.  Feedback loop will turn into a "doom loop", I

Tweet id: 1097362126216269824 | Date: 2019-02-18 | Time: 00:08:13 | Tweet: About 24.63 million shares traded. Apple Inc. (NASDAQ:AAPL) has risen 0.17% since February 17, 2018 and is uptrending. It has outperformed by 0.17% the S&P500. Some Historical AAPL News: 19/04/2018 – Apple celebrates Earth Day by debuting Daisy, $AAPL | Replies: 1
Tweet id: 1097362124643450880 | Date: 2019-02-18 | Time: 00:08:13 | Tweet: Boston Family Office Llc who had been investing in Apple Inc. for a number of months, seems to be less bullish one the $803.58 billion market cap company. The stock decreased 0.22% or $0.38 during the last trading session, reaching $170.42. $AAPL | Replies: 1
Tweet id: 1097361170359558144 | Date: 2019-02-18 | Time: 00:04:25 | Tweet: Boston Family Office Llc decreased its stake in Apple Inc. (AAPL) by 5.6% based on its latest 2018Q3 regulatory filing with the SEC. Boston Family Office Llc sold 8,562 shares as the company’s stock declined 22.32% with the market. $AAPL | Replies: 0

Tweet id: 1096172085251096576 | Date: 2019-02-14 | Time: 17:19:25 | Tweet: my guess on the Apple activity is that Ted/Todd also had some AAPL in their portfolios and they sold their position. Likely Buffett did not sell any. Just a guess. | Replies: 2
Tweet id: 1096168548827836422 | Date: 2019-02-14 | Time: 17:05:22 | Tweet: No-news-from-@CNBC-regarding-Warren-Buffett-and-Berkshire-reducing-their-Apple-stake-this-quarter  Cut-stake-by-1.1%  #13F #Berkshire $AAPL $BRKA @jimcramer @CNBCFastMoney | Replies: 0
Tweet id: 1096151968576352258 | Date: 2019-02-14 | Time: 15:59:29 | Tweet: 3) @Apple continues to advertise prominently on the Grammys🍎  Memoji ads featuring Ariana Grande, Florida Georgia Line and Khalid were pretty cool  $AAPL  https://www.digitalmusicnews.com/2019/02/08/apple-music-memoji-ariana-grande-khalid-florida-georgia-line/ … | Replies: 1
Tweet id: 1096138734909775872 | Date: 2019-02-14 | Time: 15:06:54 | Tweet: 2-investments-by-Amazon $AMZN-in-EV-firms-in-the-past-two-week

Tweet id: 1094842310402154496 | Date: 2019-02-11 | Time: 01:15:22 | Tweet: Coastline Trust Co decreased its stake in Apple Inc (AAPL) by 11.3% based on its latest 2018Q3 regulatory filing with the SEC. Coastline Trust Co sold 5,280 shares as the company’s stock declined 22.32% with the market. | Replies: 1
Tweet id: 1094730083078729728 | Date: 2019-02-10 | Time: 17:49:25 | Tweet: Exactly!  Plus, Apple would rather you spend $1000 on an iPhone then buy $1000 in AAPL.    For economy, spending > investing. | Replies: 0
Tweet id: 1094649361093611522 | Date: 2019-02-10 | Time: 12:28:39 | Tweet: I like your optimism on #Apple watch but it would be a very long road ahead to accomplish this. Apple watch, as of now, is probably the most overhyped smart wearable device in the market. Among runners, for example, Apple watch has no chance against #Garmin etc. $AAPL $GRMN | Replies: 0
Tweet id: 1094410532210335745 | Date: 2019-02-09 | Time: 20:39:38 | Tweet: No doubt that smartphone growth is slowi

...

Tweet id: 1080820516410310656 | Date: 2019-01-03 | Time: 08:37:46 | Tweet: I wouldn't be surprise if $AAPL starts making original shows for Apple TV this year with big name actors/producers/directors to capture some of $NFLX market share | Replies: 0
Tweet id: 1080817805455433728 | Date: 2019-01-03 | Time: 08:27:00 | Tweet: @tomkeene See email on $AAPL  The End of the Game For "Larry Legend" - Apple's 'Salad Days' Are Behind It | Replies: 0
Tweet id: 1080816361926991873 | Date: 2019-01-03 | Time: 08:21:15 | Tweet: @jimcramer  Are $AAPL problems apple-centric? #ThinkBIG | Replies: 0
Tweet id: 1080815001252888578 | Date: 2019-01-03 | Time: 08:15:51 | Tweet: @jimcramer you have extremely tough job.  I bought 1/3 Aapl at 200, 1/3 @ 185, 1/3 @ 175.  I hold myself accountable.  11/27/18 Mad Money"Remember, Apple has the world's biggest buyback and next week I bet you they will be in there repurchasing this stock right alongside you." | Replies: 0
Tweet id: 1080812263555051521 | Date: 2019-01

Tweet id: 1080694888247640064 | Date: 2019-01-03 | Time: 00:18:34 | Tweet: In a saturated market and with slower upgrade cycles, Apple’s strategy has been to grow profits via share gains + higher ASPs. If you think they lied, you weren’t paying attention. $AAPL Strat is transparent, but it doesn’t work in this economy and with minor YOY iPhone upgrades. | Replies: 0
Tweet id: 1080677006642163713 | Date: 2019-01-02 | Time: 23:07:31 | Tweet: I agree, but I think they are aware & will respond. We’ll just have to wait and see. I think the stock market is overreacting. You own an Apple made product that spans 5 different categories and I know you’re not an AAPL fanboy. That’s impressive. | Replies: 0
Tweet id: 1080676981845442561 | Date: 2019-01-02 | Time: 23:07:25 | Tweet: When Tim Cook took Apple to $1 trillion MC, WS praised his success.   When Q1 guidance cut, WS wants him out for lack of innovation.   Just to show you how brutal people can be when dealing with money. $AAPL | Replies: 1

Tweet id: 1080623471338680321 | Date: 2019-01-02 | Time: 19:34:47 | Tweet: It’s boils down to a greedy Tim & Apple / @AppleSupport -Not providing the masses with a reachable; Reasonably-priced refreshed #iPhoneSE - Just like they tried w/ending #iPhone5S  Gross, false pious culture with change needed ! 🧐 @WarrenBuffett $AAPL $QQQ $VXX @Carl_C_Icahn | Replies: 0
Tweet id: 1080620721498279936 | Date: 2019-01-02 | Time: 19:23:51 | Tweet: Geeez Josh, $84,000,000,000 in the quarter for Apple is something your pissed about??? I’ll keep my #AAPL and will continue to add to it the next 6 month. Thanks 🤔😊🇺🇸 @CNBCFastMoney @SquawkCNBC @CNBC @petenajarian | Replies: 0
Tweet id: 1080618174457040897 | Date: 2019-01-02 | Time: 19:13:44 | Tweet: Apple organic demand is not slowing (excluding macroeconomic issues).The bigger you get the harder it is to keep up with growth %, nonetheless your re still growing.   Big $$$ uses apple products.   AAPL seems to be more focused on growing the services (iClou

Tweet id: 1080597974390956032 | Date: 2019-01-02 | Time: 17:53:28 | Tweet: 🍎 Apple Incorporated - #WallStreet NASDAQ #AAPL $AAPL #iPhone #CongoCurse Hidden Slavery.  https://twitter.com/tim_cook/status/1033714083894243328 … https://twitter.com/Occupy007/status/1039296373764452352 … | Replies: 0
Tweet id: 1080596958329229312 | Date: 2019-01-02 | Time: 17:49:26 | Tweet: Moral of this thread: A slowdown in China is largely the biggest reason for Apple's reduced forecast, and other reasons are unforced errors on Apple's part, no matter how much they try to blame unforeseen circumstances.  More on $AAPL:  https://on.mktw.net/2QjLiBZ  | Replies: 0
Tweet id: 1080596702355169283 | Date: 2019-01-02 | Time: 17:48:25 | Tweet: @GuyAdami SO $aapl sold OFF from $210 because people were afraid of LOWER iPhone sales. OK now Apple says ur right so we sell off AGAIN???????  @CNBCJosh @jonfortt | Replies: 0
Tweet id: 1080596295331598337 | Date: 2019-01-02 | Time: 17:46:47 | Tweet: It seems the customers 

Tweet id: 1080584458376499200 | Date: 2019-01-02 | Time: 16:59:45 | Tweet: analyst on CNBC (sell rating on $AAPL)... "Cook needs to be fired", "Apple is going out of business", "the next Iphone is an Ipod", "there's 0 excitement"... LOL | Replies: 1
Tweet id: 1080584077311438848 | Date: 2019-01-02 | Time: 16:58:14 | Tweet: @jlebenthal @ScottWapnerCNBC Hey I like you and love your morning abow with Cramer but you were busting Jims balls about buying Apple now or 7 bucks lower today. He just proved his point why he was waiting. Lower AAPL guidance. Stand your ground Jim. GREAT CALL!!! | Replies: 0
Tweet id: 1080583529430282240 | Date: 2019-01-02 | Time: 16:56:04 | Tweet: Apple off 7.5% on the news. $AAPL  pic.twitter.com/ftJXqvaDeH | Replies: 0
Tweet id: 1080583131936231424 | Date: 2019-01-02 | Time: 16:54:29 | Tweet: The most important part of today’s @Apple announcement is getting left out of the news: “Finally, we also expect to report a new all-time record for Apple’s earnings per sh

In [7]:
twint.output.panda.Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username',
       'name', 'day', 'hour', 'link', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [8]:
twint.output.panda.Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username',
       'name', 'day', 'hour', 'link', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [9]:
#turning our tweet information into a pandas dataframe

tweets = twint.output.panda.Tweets_df
tweets.head(1)

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1100848143451807745,1041770783792455680,1551297624000,2019-02-27 15:00:24,-0400,,Official iOS 12 adoption stats as published by...,[#peerless],[$aapl],740928303020802048,...,,,,,,"[{'user_id': '740928303020802048', 'username':...",,,,


### Yahoo Data Aggregation

Now for stock data. We will need to pull 30 days of stock data as our target variable. We will use Yahoo Finance to do so. 

In [10]:
import yfinance as yf

# Get the data
stock_data = yf.download(tickers="AAPL",start="2019-01-01", end="2019-02-28")

# Print the data
print(stock_data.tail())

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-02-21  171.800003  172.369995  170.300003  171.059998  168.394089   
2019-02-22  171.580002  173.000000  171.380005  172.970001  170.274307   
2019-02-25  174.160004  175.869995  173.949997  174.229996  171.514664   
2019-02-26  173.710007  175.300003  173.169998  174.330002  171.613113   
2019-02-27  173.210007  175.000000  172.729996  174.869995  172.144699   

              Volume  
Date                  
2019-02-21  17249700  
2019-02-22  18913200  
2019-02-25  21873400  
2019-02-26  17070200  
2019-02-27  27835400  


In [11]:
stock_data = pd.DataFrame(stock_data)
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,154.889999,158.850006,154.229996,157.919998,154.794983,37039700
2019-01-03,143.979996,145.720001,142.000000,142.190002,139.376251,91312200
2019-01-04,144.529999,148.550003,143.800003,148.259995,145.326126,58607100
2019-01-07,148.699997,148.830002,145.899994,147.929993,145.002686,54777800
2019-01-08,149.559998,151.820007,148.520004,150.750000,147.766861,41025300


In [12]:
stock_data.shape

(39, 6)

In [13]:
dfs = stock_data
dfs.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,154.889999,158.850006,154.229996,157.919998,154.794983,37039700
2019-01-03,143.979996,145.720001,142.000000,142.190002,139.376251,91312200
2019-01-04,144.529999,148.550003,143.800003,148.259995,145.326126,58607100
2019-01-07,148.699997,148.830002,145.899994,147.929993,145.002686,54777800
2019-01-08,149.559998,151.820007,148.520004,150.750000,147.766861,41025300


# Scrub and Combine Data

### Clean Twitter Data

Here we will build a function to remove and clean unnessary text from our twitter data. All of these items listed below make it more difficult for our model to understand the sentiment/meaning of the tweets we are going to run.  

In [14]:
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet
# ______________________________________________________________
# clean dataframe's text column
tweets['tweet'] = tweets['tweet'].apply(processTweet)
# preview some cleaned tweets
tweets.head(1)

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1100848143451807745,1041770783792455680,1551297624000,2019-02-27 15:00:24,-0400,,official ios adoption stats published apple th...,[#peerless],[$aapl],740928303020802048,...,,,,,,"[{'user_id': '740928303020802048', 'username':...",,,,


In [15]:
dft['tweet'].str.replace('[{}]'.format(string.punctuation), '')

NameError: name 'dft' is not defined

There are a number of columns that are intuitivley not useful for us to keep in our diagram. So let's condense of our data

In [ ]:
dft = tweets[['date','tweet','retweet','nlikes','nreplies']]
dft.head()

Below we are going to do some EDA on the text and words in our dataset. Below is a graph listing the most popular words in our model

In [ ]:
# most common words in twitter dataset
all_words = []
for line in list(dft.tweet):
    words = line.split()
    for word in words:
        all_words.append(word.lower())
# plot word frequency distribution of first few words
plt.figure(figsize=(12,5))
plt.xticks(fontsize=13, rotation=90)
fd = nltk.FreqDist(all_words)
fd.plot(25,cumulative=False)

Stopwords are the English words which do not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. For example, the words like the, he, have etc. Such words are already captured this in corpus named corpus.

Below we are going to list out those popular stopwords which are not contributing to finding the meaning of a sentence

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')[0:20]
#listing out stop words which do not support our sentiment
stopwords_specific = [stopwords,'your','0','1012', 'and', 'for', 'the','.','that','with','you','has','was','from',"'",'&','this','are','their','its','.','']
stopwords_specific[10]

In [ ]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords_specific

tweets = dft['tweet']

test = pd.DataFrame(tweets)
test.columns = ["tweet"]

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
dft['tweet'] = dft['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(test)

Now our below graph is going to refelct a different trajectory given our removed stop words. Words like "buy, new, now, think" have more meaning to sentiment. 

In [ ]:
all_words = []
for line in list(dft.tweet):
    words = line.split()
    for word in words:
        all_words.append(word.lower())
# plot word frequency distribution of first few words
plt.figure(figsize=(12,5))
plt.xticks(fontsize=13, rotation=90)
fd = nltk.FreqDist(all_words)
fd.plot(25,cumulative=False)

In [ ]:
import string 
string.punctuation

In [ ]:
#check Jeffs slack for removing punctuation

### Sentiment Analysis

In [ ]:
dft.info()

In [ ]:
dft.shape

### Subjectivity and Polarity

Subjectivity: Subjectivity in sentiment analysis refers to expression of opinions, evaluations, feelings, and speculations and thus incorporates sentiment.

Polarity: Polarity in sentiment analysis refers to identifying sentiment orientation (positive, neutral, and negative) in written or spoken language.

In [ ]:
from textblob import TextBlob
from IPython.display import Markdown, display

In [ ]:
#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
#Create two new columns 
dft['Subjectivity'] = dft['tweet'].apply(getSubjectivity)
dft['Polarity'] = dft['tweet'].apply(getPolarity)

#Show new dataframe 
dft.head()

# EDA

### WordCloud Visualization

We will first be doing some visualizations into the types of sentences both positive and negative we have

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


In [ ]:
#Plot the worldcloud 
allwords = ' '.join([twts for twts in dft['tweet']])
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allwords)

plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

Some of the biggest words point to "will", "price", "phone", "buy", "now"

In [ ]:
#Create a function to compute the negative, neutral and positive analysis

def getAnalysis(score):
    if score < 0: 
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    elif score > 0:
        return 'Positive'
    
dft['Analysis'] = dft['Polarity'].apply(getAnalysis)
dft

In [ ]:
#Print all the positive tweets
j=1
sortedDF = dft.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
    if(sortedDF['Analysis'][i] == 'Positive'):
        print(str(j) + ') '+sortedDF['tweet'][i])
        print()
        j = j+1

In [ ]:
#Print all the negative tweets
j=1
sortedDF = dft.sort_values(by=['Polarity'], ascending=False)
for i in range(0, sortedDF.shape[0]):
    if(sortedDF['Analysis'][i] == 'Negative'):
        print(str(j) + ') '+ sortedDF['tweet'][i])
        print()
        j = j+1

In [ ]:
#plot the polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range(0, dft.shape[0]):
    plt.scatter(dft['Polarity'][i], dft['Subjectivity'][i], color= 'Blue')
    
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Sujectivity')
plt.show()
    

In [ ]:
import seaborn as sns
#Bar graph to visualize sentiment

y = dft["Analysis"].value_counts()
sns.barplot(y.index, y.values)
y

Predominately positive correlation of tweets

In [ ]:
from pylab import rcParams

#Pie Chart shows a better visualization

sizes = dft['Analysis'].value_counts(sort = True)
colors = ["blue","orange"] 
rcParams['figure.figsize'] = 5,5
# Plot
plt.pie(sizes, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Percentage of Tweet Sentiment')
plt.show()

In [ ]:
# Join all the tweets into a single string
all_words_string = ' '.join([str(dft.tweet) for tweet in dft.tweet])

# Make a list of all the words in the dataframe
all_words_list = all_words_string.split()

# Plot word frequency distribution of first few words

plt.figure(figsize=(12,5))
plt.xticks(fontsize=13, rotation=90)
freq_dist = nltk.FreqDist(all_words_list)
freq_dist.plot(25,cumulative=False)

In [ ]:
Breaking it down by total number of most popular words

In [ ]:
word_freq = nltk.FreqDist(all_words_list)
freq_df = pd.DataFrame({'Word': list(word_freq.keys()), 'Count': list(word_freq.values())}).sort_values(by=['Count'], ascending=False)
freq_df.head(15)

### Combine Datasets

Now that we have our two datasets finalized, its time to combine them in preparation for our time series. We will do so with an inner join

In [ ]:
#Average subjectivity and polarity scores based on day rather than time
dft1 = dft[['date','Subjectivity','Polarity','Analysis']]
dft1.head()

In [ ]:
dft1.info()

In [ ]:
dft1.date = pd.to_datetime(dft1.date)

In [ ]:
#Average subjectivity and polarity scores based on day rather than time. Remove time. 
dft1 = dft1.resample('D', on='date').mean()
dft1.head()

In [ ]:
dfs.head()

Creating a new column below which identifies Open_Close Shift. The pupose of this is to more concretely see where the stock price went from start to end of the day. This gives us a eaiser visual into its comparision with polarity and subjectivity 

In [ ]:
# Create column to view the change in open prices from prior days close
# Create a column that labels whether the open-close difference was positive or negative

dfs['Open_Close_Shift'] = dfs.Close - dfs.Open
dfs['Positive_Negative'] = dfs['Positive_Negative'] = dfs.Open_Close_Shift.apply(lambda x: 1 if (x > 0) else (-1 if x < 0 else 0))
dfs.head()

In [ ]:
df = pd.merge(dfs, dft1, how='inner', left_index=True, right_index=True)
df.head()

In [ ]:
#adding Analysis back into the dataframe
df['Analysis'] = df['Polarity'].apply(getAnalysis)
df.head()

In [ ]:
df.index.names = ['Date']
df.head()

In [ ]:
#drop unneccary columns
df.drop(['Open','High','Low','Close','Adj Close','Volume'], axis=1, inplace=True)
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

Visualization to show that negative tweets were the only places where we saw a negative analysis. This is a good sign for our model and correlation

In [ ]:
#Taking a look a visual look at sentiment and change in stock price
df.groupby(["Positive_Negative", "Analysis"]).size().unstack().plot(kind='bar', stacked=True, figsize=(5,5))


### Line Graphs of our columns

Showing quick line graphs for each of our columns

In [ ]:
#open_close_shift
df['Open_Close_Shift'].plot(figsize=(12,6), linewidth=2, fontsize=10)
plt.legend(ncol=2)
plt.xlabel("Date",fontsize=12)
plt.ylabel("Stock Price Change",fontsize=12)
plt.title("Apple Stock Price Over Time", fontsize=16);


In [ ]:
#Subjectivity 
df['Subjectivity'].plot(figsize=(12,6), linewidth=2, fontsize=10)
plt.legend(ncol=2)
plt.xlabel("Date",fontsize=12)
plt.ylabel("Subjectivity Rating",fontsize=12)
plt.title("Twitter Subjectivity Over Time", fontsize=16);

In [ ]:
#Polarity
df['Polarity'].plot(figsize=(12,6), linewidth=2, fontsize=10)
plt.legend(ncol=2)
plt.xlabel("Date",fontsize=12)
plt.ylabel("Polarity Rating",fontsize=12)
plt.title("Twitter Polarity Over Time", fontsize=16);

In [ ]:
df1 = df.copy()
df1["Open_Close_Shift"]=((df1["Open_Close_Shift"]-df1["Open_Close_Shift"].min())/(df1["Open_Close_Shift"].max()-df1["Open_Close_Shift"].min()))*.6
df1["Polarity"]=((df1["Polarity"]-df1["Polarity"].min())/(df1["Polarity"].max()-df1["Polarity"].min()))*.6

In [ ]:
df1.drop(['Positive_Negative','Analysis','Subjectivity'], axis=1, inplace=True)

In [ ]:
df1.to_csv('stocks-polarity.csv',index=True)

Below we will explore the trajectory of Polarity vs Subjectivity in predicting stock price. Which one follows a more silmilar pattern

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')
series = read_csv('stocks-polarity.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# summarize first few rows
print(series.head())
# line plot
series.plot()
plt.title("Polarity || Stock Trend", fontsize=16);
pyplot.show()
plt.legend


In [ ]:
df2=df.copy()
df2["Open_Close_Shift"]=((df2["Open_Close_Shift"]-df2["Open_Close_Shift"].min())/(df2["Open_Close_Shift"].max()-df2["Open_Close_Shift"].min()))*.6
df2["Subjectivity"]=((df2["Subjectivity"]-df2["Subjectivity"].min())/(df2["Subjectivity"].max()-df2["Subjectivity"].min()))*.6

In [ ]:
df2.drop(['Polarity','Positive_Negative','Analysis'], axis=1, inplace=True)

In [ ]:
df2.to_csv('stocks-subjectivity.csv',index=True)

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')
series = read_csv('stocks-subjectivity.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# summarize first few rows
print(series.head())
# line plot
series.plot()
plt.title("Subjectivity || Stock Trend", fontsize=16);
pyplot.show()
plt.legend

Ultimately it is visually clear that Subjectivity more closely follows the curve of stock price than Polarity. We will use Subjectiviity as our predictive columnd alongside change in stock price below

# Time Series Pediction Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN,LSTM
from keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib as mpl
import os

In [ ]:
df.drop(['Positive_Negative','Analysis','Polarity'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

We are going to use a multivariate time series model to account for both subjectivity and stock price. Allows us to use two inputs and spit out 1 output (predicted stock price). 


In [ ]:
# multivariate data preparation
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
in_seq1 = array(df['Subjectivity'])
in_seq2 = array(df['Open_Close_Shift'])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 

Running the sample above first prints the shape of the X and y components. We can see that the X component has a three-dimensional structure.

The first dimension is the number of samples, in this case 37. The second dimension is the number of time steps per sample, in this case 3, the value specified to the function. Finally, the last dimension specifies the number of parallel time series or the number of variables, in this case 2 for the two parallel series.

We can then see that the input and output for each sample is printed, showing the three time steps for each of the two input series and the associated output for each sample.

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# define input sequence
in_seq1 = array(df['Subjectivity'])
in_seq2 = array(df['Open_Close_Shift'])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(25, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[.39, .069], [.32, .61], [.17, 1.6]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

When making a prediction, the model expects three time steps for two input time series.

We can predict the next value in the output series proving the input values of subjectivity and stock price for the past 3 days



In [ ]:
X.shape

In [ ]:
X[-1].reshape(1,3,2)

In [ ]:
x_input = array([[.2, .1], [-.4, -.3], [.1, -1]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Running the past three inputs above, fits the model, and makes a prediction.


Now lets create a train and test set check out model

In [ ]:
train_x = X[:30]

train_y = y[:30]

test_x = X[30:]

test_y = y[30:]

In [ ]:
model_history = model.fit(train_x, train_y, epochs=200, verbose=1,validation_data=[test_x,test_y])
model_history
#loss ie MSE. Good that is getting smaller overtime 

Our loss and val_loss (MSE) is decreasing as our Epochs move forward. This is a good sign that our model is accurate.

In [ ]:
model.predict(X[-1].reshape(1,3,2))

Our final product will be an input structure where we can plot in the past 3 days of inputs and it will spit our a predicted stock price

In [ ]:
yesterday_stock = float(input("stock price change from yesterday = "))
yesterday_subjectivity = float(input("subjectivity  from yesterday = "))
two_days_ago_stock = float(input("stock price change from 2 days ago = "))
two_days_ago_subjectivity = float(input("subjectivity  from 2 days ago = "))
three_days_ago_stock = float(input("stock price change from 3 days ago = "))
three_days_ago_subjectivity = float(input("subjectivity  from 3 days ago = "))

x_input = array([[yesterday_stock,yesterday_subjectivity],[two_days_ago_stock, two_days_ago_subjectivity],
                [three_days_ago_stock, three_days_ago_subjectivity]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

# Conclusions

There is a correlation between tweet subjectivity and stock price. Some places I would further examine: 

1) Recurring twitter patterns. Would be great to tap into the Twitter API and pull a model that updates with daily tweets

2) Would be keen to run this model on other companies and see what the patterns look like

3) Intertwine other factors into predicting stock price such as newspaper headlines or specific company data 

